In [1]:
import numpy as np
from cosmoTransitions import generic_potential as gp
from cosmoTransitions import pathDeformation as pd
from scipy import interpolate, optimize

In [4]:
GF = 1.16637e-05 
v = 1/np.sqrt(np.sqrt(2)*GF)
vR=10**5
aB = np.exp(3.91)
aF = np.exp(1.14)

In [201]:
class model(gp.generic_potential):
    def init(self, lR, A, muS, g1,g2,yt):
        self.yt=yt
        self.g=g2
        self.gy=g1
        self.A = A
        self.lR = lR
        self.muS = muS
        self.cs = 2/3
        self.muH2 = self.lR * vR**2
        self.mWsq = 0.25*self.g**2*vR**2
        self.mZsq = 0.25*(self.g**2 + self.gy**2) * vR**2
        self.mtsq = 0.5*self.yt**2 * vR**2
        self.D = (3*self.g**2 + self.gy**2 + 4*self.yt**2)/16.
        self.B = 3*(2*self.mWsq**2 + self.mZsq**2 - 4*self.mtsq**2)/(64*np.pi**2 * vR**4)
        self.E = (2*self.g**3 + (self.g**2 + self.gy**2)**(3/2))/(32*np.pi)
        self.T0 = ((self.muH2 - 4*self.B * vR**2)/(2*self.D))**0.5
        self.Ndim = 2
        self.Tmax = 50000
        self.Deff = self.D - self.cs * self.A**2 / (4.0 * self.muS**2)
        self.T01d = ((self.muH2 - 4*self.B * vR**2 - 0.5*vR**2 * self.A**2/self.muS**2)/(2*self.Deff))**0.5
        self.Tc = False

    def lR_T(self,T):
        T = np.asanyarray(T, dtype=float)
        T2 = (T*T) + 1e-100
        correction = 2 * self.mWsq**2 * np.log(self.mWsq/(aB * T2))
        correction += self.mZsq**2 * np.log(self.mZsq/(aB*T2))
        correction += - 4*self.mtsq**2 * np.log(self.mtsq/(aF*T2))
        correction = correction*3/(16*np.pi**2 * vR**4)
        return self.lR - correction

    def approxZeroTMin(self):
        return [np.array([vR,0]),np.array([-vR,0])]

    def resum(self, X, T):
        T = np.asanyarray(T, dtype=float)
        X = np.asanyarray(X, dtype=float)
        T2 = (T * T) + 1e-100
        phi1 = X[..., 0]
        Sq = -88*self.g**2 * self.gy**2 *T2*(9*phi1**2 + 44*T2) + (self.g**2*(3*phi1**2 + 22*T2) + self.gy**2 * (3*phi1**2 + 44*T2))**2
        mZL = (3 * self.g**2 * phi1**2 + 3 * self.gy**2 * phi1**2 + 22 * self.g**2 * T2 + 44 * self.gy**2 * T2 + np.sqrt(Sq))/24.
        mBL = (3 * self.g**2 * phi1**2 + 3 * self.gy**2 * phi1**2 + 22 * self.g**2 * T2 + 44 * self.gy**2 * T2 - np.sqrt(Sq))/24.
        mWL = 11*self.g**2 * T2/6 + 0.25*self.g**2 * phi1**2
        resum = 2 * ( (0.25*self.g**2 * phi1**2)**(3/2) - mWL**(3/2))
        resum += (0.25*(self.g**2 + self.gy**2)*phi1**2)**(3/2) - mZL**(3/2)
        resum += - abs(mBL)**(3/2)
        return resum*T/(12*np.pi)
    
    def Vtot(self, X, T, include_ratiation=True):
        T = np.asanyarray(T, dtype=float)
        X = np.asanyarray(X, dtype=float)
        T2 = (T * T) + 1e-100
        phi1 = X[..., 0]
        phi2 = X[..., 1] 
        y = self.D * (T2 - self.T0**2) * phi1**2 - self.E*T*phi1**3 + 0.25 * self.lR_T(T) * phi1**4
        y += (0.5*self.muS**2 * phi2**2 - 0.5 * self.A * (phi1**2 + self.cs * T2 - vR**2) * phi2)
        y += self.resum(X,T)
        return y

    def Spath(self, X, T):
        X = np.asanyarray(X)
        T = np.asanyarray(T)
        phi1 = X[..., 0]
        T2 = (T * T) + 1e-100
        return 0.5 * self.A * (phi1**2 + self.cs * T2 - vR**2) / self.muS**2

    def find_Tc(self):
        Tmax = self.Tmax
        Tmin = 1000
        for i in range(0,20):
            Ttest = (Tmax + Tmin)/2
            truevev = self.findMinimum([0.3*vR, self.Spath([0.3*vR], Ttest)], Ttest)
            if truevev[0] >= 1:
                Tmin = Ttest
            else:
                Tmax = Ttest
        self.Tc = Ttest

    def tunneling_at_T(self, T):
        if self.Tc == False:
            self.find_Tc()

        assert T < self.Tc

        def V_(x, T=T, V=self.Vtot):
            return V(x, T)

        def dV_(x, T=T, dV=self.gradV):
            return dV(x, T)

        truevev = self.findMinimum([0.3*vR, self.Spath([0.3*vR], T)], T)
        # tobj = pd.fullTunneling([self.findMinimum(T=T),[0,self.Spath([0],T)]],V_,dV_)
        tobj = pd.fullTunneling(
            [
                truevev,
                [1e-100, self.Spath([1e-100], T)],
            ],
            V_,
            dV_,
        )
        return tobj
    

In [202]:
m=model(0.0517831, 5.644776110372951, 17.992222319658012, 0.371575, 0.616563, 0.713062) # mS=15, sin=0.08

In [143]:
m.Vtot([vR,m.Spath([vR],T=40000)],T=40000)-m.Vtot([0,m.Spath([0],T=40000)],T=40000)

2.3335324827589084e+18

In [199]:
m.find_Tc()

In [200]:
m.Tc

13899.500846862793

In [192]:
vev = m.findMinimum([30000,m.Spath([30000],13800)],13800)
print(vev)

[ 2.20277201e+04 -8.18487117e+07]


In [186]:
m.Vtot(vev,13500)

-1.2036898070565386e+18

In [232]:
m.tunneling_at_T(13769.1243567)

Maximum number of deformation iterations reached.
Deformation doesn't appear to be converging.Stopping at the point of best convergence.
Deformation doesn't appear to be converging.Stopping at the point of best convergence.
Deformation doesn't appear to be converging.Stopping at the point of best convergence.
Deformation doesn't appear to be converging.Stopping at the point of best convergence.


PotentialError: ('Barrier height is not positive, does not exist.', 'no barrier')